In [1]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [4]:
myAPIkey = "yourAPI"
quandl.ApiConfig.api_key = myAPIkey

In [5]:
start_date = "2014-12-01"
end_date = "2022-04-01"

In [133]:
def get_adj_close(ticker,start_date,end_date):
    
    df = quandl.get_table('QUOTEMEDIA/PRICES', ticker=ticker,date = {'gte':start_date, 'lte':end_date}) 
        
    df = df.sort_values(by='date')

    df = df[['date','adj_close']]
    df = df.rename(columns={'adj_close': ticker+'_adj_close'})
    
    df = df.set_index(['date'])

    return df


def fill_zacks_fc(ticker,start_date,end_date):
    
    df_fc = quandl.get_table('ZACKS/FC',ticker = ticker,per_end_date = {'gte':start_date,'lte':end_date})
    
    df_fc = df_fc[df_fc.per_type == 'Q']
    df_fc = df_fc[['per_end_date','filing_date','eps_diluted_net','basic_net_eps','net_lterm_debt','tot_lterm_debt']]
    df_fc = df_fc.sort_values(by='per_end_date')
    
    df_fc['date'] = df_fc['per_end_date']
    df_fc = df_fc.set_index(['date'])
    
    return df_fc


def fill_zacks_fr(ticker,start_date,end_date):
    
    df_fr = quandl.get_table('ZACKS/FR',ticker = ticker,per_end_date = {'gte':start_date,'lte':end_date})
    
    df_fr = df_fr[df_fr.per_type == 'Q']
    df_fr = df_fr[['per_end_date', 'tot_debt_tot_equity','ret_invst']]
    df_fr = df_fr.sort_values(by='per_end_date')
    
    df_fr['date'] = df_fr['per_end_date']
    df_fr = df_fr.set_index(['date'])
    
    df_fr = df_fr[['tot_debt_tot_equity','ret_invst']]
       
    return df_fr


def fill_zacks_mktv(ticker,start_date,end_date):
    
    df_mktv = quandl.get_table('ZACKS/MKTV',ticker = ticker,per_end_date = {'gte':start_date,'lte':end_date})
    
    df_mktv = df_mktv[['per_end_date', 'mkt_val']]
    df_mktv = df_mktv.sort_values(by='per_end_date')
    
    df_mktv['date'] = df_mktv['per_end_date']
    df_mktv = df_mktv.set_index(['date'])
    
    df_mktv = df_mktv[['mkt_val']]
    
    return df_mktv


def fill_zacks_shrs(ticker,start_date,end_date):
    
    df_shrs = quandl.get_table('ZACKS/SHRS',ticker = ticker,per_end_date = {'gte':start_date,'lte':end_date})
    
    df_shrs = df_shrs[['per_end_date', 'shares_out']]
    df_shrs = df_shrs.sort_values(by='per_end_date')
    
    df_shrs['date'] = df_shrs['per_end_date']
    df_shrs = df_shrs.set_index(['date'])
    
    df_shrs = df_shrs[['shares_out']]
    
    return df_shrs

In [7]:
def generate_zacks_data(ticker,start_date,end_date):
    
    df_fc = fill_zacks_fc(ticker,start_date,end_date)
    df_fr = fill_zacks_fr(ticker,start_date,end_date)
    df_mktv = fill_zacks_mktv(ticker,start_date,end_date)
    df_shrs = fill_zacks_shrs(ticker,start_date,end_date)
    
    df = df_fc.join(df_fr)

    df = df.join(df_mktv)
    df = df.join(df_shrs)
    
    return df

In [103]:
df_shrs = fill_zacks_shrs('LLY','2015-01-01','2022-01-31')

In [104]:
df_zack = generate_zacks_data('LLY','2015-01-01','2022-01-31')

In [105]:
df_price = get_adj_close('LLY','2015-01-01','2022-01-31')

In [106]:
df = df_price.join(df_zack,how='outer')

In [107]:
# df['LLY_adj_close'] =  df['LLY_adj_close'].fillna(method='bfill')
df['LLY_adj_close'] =  df['LLY_adj_close'].fillna(method='ffill')

In [108]:
df[df['filing_date'].notna()]

,LLY_adj_close,per_end_date,filing_date,eps_diluted_net,basic_net_eps,net_lterm_debt,tot_lterm_debt,tot_debt_tot_equity,ret_invst,mkt_val,shares_out
date,,,,,,,,,,,
2015-03-31,61.840466,2015-03-31,2015-04-30,0.50,0.50,2182.0,7415.3,0.5500,2.3679,80714.88,1111.01
2015-06-30,71.553956,2015-06-30,2015-07-30,0.56,0.56,2505.4,7988.6,0.5440,2.6474,92758.23,1111.01
2015-09-30,72.155475,2015-09-30,2015-10-30,0.75,0.75,2504.0,8069.6,0.5301,3.4306,92773.72,1108.54
2015-12-31,73.100618,2015-12-31,2016-02-19,0.45,0.45,2499.0,7972.4,0.5468,2.1203,93366.82,1108.08
2016-03-31,62.909043,2016-03-31,2016-04-29,0.41,0.41,NaN,7477.6,0.5407,1.9554,79534.33,1104.49
2016-06-30,69.257001,2016-06-30,2016-07-28,0.71,0.71,1206.5,8685.5,0.6367,3.2033,86978.59,1104.49
2016-09-30,71.031828,2016-09-30,2016-10-28,0.73,0.73,1206.4,8707.3,0.5987,3.1986,88603.02,1103.95
2016-12-31,65.517096,2016-12-31,2017-02-21,0.73,0.73,1206.4,8367.8,0.7319,3.4381,81195.52,1103.95
2017-03-31,75.424735,2017-03-31,2017-05-01,-0.10,-0.10,-630.2,7637.5,0.7265,-0.5096,92802.77,1103.35


In [109]:
df_tot_debt_tot_equity = df[['filing_date','tot_debt_tot_equity']].dropna()
# df_tot_debt_tot_equity.reset_index(inplace = True)
df_tot_debt_tot_equity

,filing_date,tot_debt_tot_equity
date,,
2015-03-31,2015-04-30,0.5500
2015-06-30,2015-07-30,0.5440
2015-09-30,2015-10-30,0.5301
2015-12-31,2016-02-19,0.5468
2016-03-31,2016-04-29,0.5407
2016-06-30,2016-07-28,0.6367
2016-09-30,2016-10-28,0.5987
2016-12-31,2017-02-21,0.7319
2017-03-31,2017-05-01,0.7265


In [110]:
# df_tot_debt_tot_equity = df_tot_debt_tot_equity.rename(columns={'filing_date': 'date'})
# df_tot_debt_tot_equity = df_tot_debt_tot_equity.set_index(['date'])
# df_tot_debt_tot_equity

In [111]:
quarter_date = list(pd.date_range(start='2015-03-31', periods=28, freq='3M'))
quarter_date = [date_obj.strftime('%Y-%m-%d') for date_obj in quarter_date]
# quarter_date 

In [112]:
# df_quarter_adj_close = df.loc[quarter_date]['LLY_adj_close']
# df_quarter_adj_close

In [113]:
df_tot_debt_tot_equity['quarter_adj_close'] = df.loc[quarter_date]['LLY_adj_close']

In [114]:
df_tot_debt_tot_equity

,filing_date,tot_debt_tot_equity,quarter_adj_close
date,,,
2015-03-31,2015-04-30,0.5500,61.840466
2015-06-30,2015-07-30,0.5440,71.553956
2015-09-30,2015-10-30,0.5301,72.155475
2015-12-31,2016-02-19,0.5468,73.100618
2016-03-31,2016-04-29,0.5407,62.909043
2016-06-30,2016-07-28,0.6367,69.257001
2016-09-30,2016-10-28,0.5987,71.031828
2016-12-31,2017-02-21,0.7319,65.517096
2017-03-31,2017-05-01,0.7265,75.424735


In [115]:
df_tot_debt_tot_equity = df_tot_debt_tot_equity.rename(columns={'filing_date': 'date'})
df_tot_debt_tot_equity = df_tot_debt_tot_equity.set_index(['date'])
df_tot_debt_tot_equity

,tot_debt_tot_equity,quarter_adj_close
date,,
2015-04-30,0.5500,61.840466
2015-07-30,0.5440,71.553956
2015-10-30,0.5301,72.155475
2016-02-19,0.5468,73.100618
2016-04-29,0.5407,62.909043
2016-07-28,0.6367,69.257001
2016-10-28,0.5987,71.031828
2017-02-21,0.7319,65.517096
2017-05-01,0.7265,75.424735


In [116]:
df_debt_to_market_cap = df[['LLY_adj_close']].join(df_tot_debt_tot_equity)

In [117]:
df_debt_to_market_cap

,LLY_adj_close,tot_debt_tot_equity,quarter_adj_close
date,,,
2015-01-02,59.212581,NaN,NaN
2015-01-05,58.629414,NaN,NaN
2015-01-06,58.925223,NaN,NaN
2015-01-07,58.511090,NaN,NaN
2015-01-08,59.897168,NaN,NaN
...,...,...,...
2022-01-25,236.261694,NaN,NaN
2022-01-26,234.683064,NaN,NaN
2022-01-27,233.558291,NaN,NaN


In [118]:
df_debt_to_market_cap['tot_debt_tot_equity'] = df_debt_to_market_cap['tot_debt_tot_equity'].fillna(method='ffill')
df_debt_to_market_cap['quarter_adj_close'] = df_debt_to_market_cap['quarter_adj_close'].fillna(method='ffill')
df_debt_to_market_cap

,LLY_adj_close,tot_debt_tot_equity,quarter_adj_close
date,,,
2015-01-02,59.212581,NaN,NaN
2015-01-05,58.629414,NaN,NaN
2015-01-06,58.925223,NaN,NaN
2015-01-07,58.511090,NaN,NaN
2015-01-08,59.897168,NaN,NaN
...,...,...,...
2022-01-25,236.261694,2.148,227.220184
2022-01-26,234.683064,2.148,227.220184
2022-01-27,233.558291,2.148,227.220184


In [119]:
df_debt_to_market_cap['debt_to_market_cap'] = df_debt_to_market_cap['tot_debt_tot_equity']*df_debt_to_market_cap['LLY_adj_close']/df_debt_to_market_cap['quarter_adj_close']

df_debt_to_market_cap

,LLY_adj_close,tot_debt_tot_equity,quarter_adj_close,debt_to_market_cap
date,,,,
2015-01-02,59.212581,NaN,NaN,NaN
2015-01-05,58.629414,NaN,NaN,NaN
2015-01-06,58.925223,NaN,NaN,NaN
2015-01-07,58.511090,NaN,NaN,NaN
2015-01-08,59.897168,NaN,NaN,NaN
...,...,...,...,...
2022-01-25,236.261694,2.148,227.220184,2.233473
2022-01-26,234.683064,2.148,227.220184,2.218549
2022-01-27,233.558291,2.148,227.220184,2.207917


In [120]:
df_debt_to_market_cap.loc['2020-10-28']

LLY_adj_close          126.957708
tot_debt_tot_equity      3.387100
quarter_adj_close      143.113852
debt_to_market_cap       3.004730
Name: 2020-10-28 00:00:00, dtype: float64

In [123]:
df_debt_to_market_cap.loc['2020-10-29']

LLY_adj_close          128.156608
tot_debt_tot_equity      3.387100
quarter_adj_close      143.113852
debt_to_market_cap       3.033104
Name: 2020-10-29 00:00:00, dtype: float64

In [124]:
df_debt_to_market_cap.loc['2021-12-31']

LLY_adj_close          272.530716
tot_debt_tot_equity      2.148000
quarter_adj_close      227.220184
debt_to_market_cap       2.576338
Name: 2021-12-31 00:00:00, dtype: float64

In [95]:
3.004730/2.224891

1.350506609087816

In [96]:
4.755473/3.782411

1.2572597213787713

In [126]:
128.156608/126.957708

1.0094433021742957

In [127]:
4.755473/3.782411

1.2572597213787713

In [97]:
'2017-06-30' < '2017-12-29'

True

In [125]:
df_debt_to_market_cap[800:850]

,LLY_adj_close,tot_debt_tot_equity,quarter_adj_close,debt_to_market_cap
date,,,,
2018-03-05,71.541205,1.1696,77.201629,1.083845
2018-03-06,71.071630,1.1696,77.201629,1.076731
2018-03-07,71.301814,1.1696,77.201629,1.080218
2018-03-08,72.636882,1.1696,77.201629,1.100444
2018-03-09,73.299812,1.1696,77.201629,1.110488
2018-03-12,72.774992,1.1696,77.201629,1.102537
2018-03-13,72.876273,1.1696,77.201629,1.104071
2018-03-14,72.268587,1.1696,77.201629,1.094865
2018-03-15,72.507978,1.1696,77.201629,1.098491


In [100]:
df_debt_to_market_cap[150:200]

,LLY_adj_close,tot_debt_tot_equity,quarter_adj_close,debt_to_market_cap
date,,,,
2015-08-07,70.508372,0.5301,72.155475,0.517999
2015-08-10,71.571097,0.5301,72.155475,0.525807
2015-08-11,71.596808,0.5301,72.155475,0.525996
2015-08-12,71.888200,0.5301,72.155475,0.528136
2015-08-13,71.586439,0.5301,72.155475,0.525920
2015-08-14,72.345153,0.5301,72.155475,0.531493
2015-08-17,71.965796,0.5301,72.155475,0.528707
2015-08-18,71.896822,0.5301,72.155475,0.528200
2015-08-19,72.224449,0.5301,72.155475,0.530607


In [132]:
def fill_zacks_fc(ticker,start_date,end_date):
    
    df_fc = quandl.get_table('ZACKS/FC',ticker = ticker,per_end_date = {'gte':start_date,'lte':end_date})
    
    # print(list(df_fc.columns))
    df_fc = df_fc[df_fc.per_type == 'Q']
    # df_fc = df_fc.drop_duplicates(subset=['per_end_date', 'filing_date'])
    df_fc = df_fc[['per_end_date','filing_date','eps_diluted_net','basic_net_eps','net_lterm_debt','tot_lterm_debt']]
    # df_fc = df_fc.drop_duplicates(subset=['per_end_date', 'filing_date'])
    df_fc = df_fc.sort_values(by='per_end_date')
    
    df_fc['date'] = df_fc['per_end_date']
    df_fc = df_fc.set_index(['date'])
    
    return df_fc
    
df1 = fill_zacks_fc('LLY','2015-01-01','2022-01-31')
df1

,per_end_date,filing_date,eps_diluted_net,basic_net_eps,net_lterm_debt,tot_lterm_debt
date,,,,,,
2015-03-31,2015-03-31,2015-04-30,0.50,0.50,2182.0,7415.3
2015-06-30,2015-06-30,2015-07-30,0.56,0.56,2505.4,7988.6
2015-09-30,2015-09-30,2015-10-30,0.75,0.75,2504.0,8069.6
2015-12-31,2015-12-31,2016-02-19,0.45,0.45,2499.0,7972.4
2016-03-31,2016-03-31,2016-04-29,0.41,0.41,NaN,7477.6
2016-06-30,2016-06-30,2016-07-28,0.71,0.71,1206.5,8685.5
2016-09-30,2016-09-30,2016-10-28,0.73,0.73,1206.4,8707.3
2016-12-31,2016-12-31,2017-02-21,0.73,0.73,1206.4,8367.8
2017-03-31,2017-03-31,2017-05-01,-0.10,-0.10,-630.2,7637.5
